<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Diverse Plugins for Decision Making

Decision Intelligence applied in this module:  
* Using the power of Collective Intelligence (wisdom & information from multiple experts) 
* Using data statistics & analytics as source information for a decision   
* Using Probability as a quantitative approach to validate decisions with Machine Learning  

> "Probability is the language of uncertainty, and it's the only language we have for communicating what we know and don't know."
>
> -- <cite>Nate Silver (famous statistician and super-forecaster)</cite> 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.5"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.5"
#r "nuget: Microsoft.SemanticKernel, 1.54"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.54-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.54-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"
#r "nuget: Microsoft.ML, 4.0.2"
#r "nuget: Microsoft.ML.FastTree, 4.0.2"
#r "nuget: Microsoft.Extensions.ML, 4.0.2"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 9.0.5 Microsoft.Extensions.Configuration.Json, 9.0.5 Microsoft.Extensions.ML, 4.0.2 Microsoft.ML, 4.0.2 Microsoft.ML.FastTree, 4.0.2 Microsoft.SemanticKernel, 1.54.0 Microsoft.SemanticKernel.Plugins.Core, 1.54.0-preview Microsoft.SemanticKernel.Plugins.Web, 1.54.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

// Import the BaseballHallOfFamePlugin.cs file
#!import ../Classes/BaseballHallOfFamePlugin.cs

// Add the native plugin with the three native functions to the semantic kernel
semanticKernel.ImportPluginFromType<BaseballHallOfFamePlugin>();

Using Azure OpenAI Service


### Step 2 - Introduce Sports Decision Scenario 

<img width ="1024x" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SportsDecisionScenario.png">

### Step 3 - Background Information

Execute the cell below to understand what type of collective information will be used to help the GenAI model make the decsion. In addition to the internet (knowledge graph) shown in previous modules, historical baseball statistic and Machine Learning models exposing a quantitative probability are available.  

In [3]:
// Background Information on what is happening & components

// 1) Directory of the historical baseball data statistics
string filePathMLBBaseballBatters = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Data/MLBBaseballBattersPositionPlayers.csv";
// Read the batters data from the CSV file
var batters = File.ReadAllLines(filePathMLBBaseballBatters)
                        .Skip(1)
                        .Select(v => MLBBaseballBatter.FromCsv(v))
                        .ToList();

// 2) A custom decision rules engine crafted by a Machine Learning model
// Machine Learing Model and Prediction Engine                       
string modelPath = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Models/InductedToHoF-GeneralizedAdditiveModels.mlnet";
var mlContext = new MLContext(seed: 100);
ITransformer loadedModel;
DataViewSchema schema;

using (var stream = File.OpenRead(modelPath))
{
    loadedModel = mlContext.Model.Load(stream, out schema);
}
TransformerChain<ITransformer> transfomerChain = (TransformerChain<ITransformer>)loadedModel;

var predEngineInductedToHallOfFame = mlContext.Model.CreatePredictionEngine<MLBBaseballBatter, MLBHOFPrediction>(transfomerChain);

// 3) Predict if Mike Trout will be inducted to the Hall of Fame
var mikeTroutStats = batters.Where(b => b.FullPlayerName == "Mike Trout").FirstOrDefault();
Console.WriteLine("Mike Trout Stats From Historical Data (end of 2023 season):");
Console.WriteLine(mikeTroutStats.ToString());
var prediction = predEngineInductedToHallOfFame.Predict(mikeTroutStats);
Console.WriteLine("--------------------");
Console.WriteLine("Mike Trout Probabibitiy of being inducted to Hall Of Fame From ML Model:");
Console.WriteLine($"{prediction.Probability:P2}");

Mike Trout Stats From Historical Data (end of 2023 season):
FullPlayerName:Mike Trout, YearPlayed:13, AB:5402
        R:1106, H:1624, Doubles:310, Triples:52, HR:368, RBI:940, SB:206,
        BattingAvg:0.301, Slg:0.582, AllStar:11,
        TB:3142, LastYearPlayed:2023, ID:troutmi01
--------------------
Mike Trout Probabibitiy of being inducted to Hall Of Fame From ML Model:
94.55%


### Step 4 - Hall of Fame Decision - Only using Web Search Results 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame using current internet information only.

**Notice the instruction in the prompt is to only use the web search results.** While there might be multiple functions available to the GenAI model via the imported plugin, the prompt instruction is to use only the single "tool" at it's disposal. Therefore, historical baseball statistics and the Machine Learning model probabilities will not be used.  

After executing the code below, notice that the GenAI model has gathered intelligence using Bing. This intelligence about Mike Trout's baseball hall of fame candidacy is provides additional intelligence context. Finally, the LLM provides the information it has gathered, cites its sources and arrives at a decision (conclusion).

In [4]:
// Use WebSearch to find information on Mike Trout

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Only use web search results to make your decision.
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the web search results, I strongly recommend that Mike Trout should be inducted into the Hall of Fame.

Multiple reputable sources highlight that Trout is considered a "virtual lock" for the Hall of Fame due to his extraordinary career achievements, including three MVP awards and numerous All-Star appearances. His statistical dominance from 2011-2020 is noted as one of the most prolific runs in baseball history, making him one of the greatest players of his generation. The consensus among analysts is not whether he will make the Hall of Fame, but rather when he will be inducted, with many asserting he is already a future Hall of Famer even if his career were to end today [4][7][5].

While there are discussions about his recent injuries and whether they might impact his candidacy, the overwhelming view is that his accomplishments already far exceed typical Hall of Fame standards, and his legacy is secure despite setbacks [3][8].

In summary, the evidence from multiple sources s

### Step 5 - Hall of Fame Decision - Only using Baseball Statistics 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame.  

**Notice the instruction in the prompt instruction is to only use the historical baseball statistics.** Running the code below will gather intelligence from the local MLBBaseballBattersPositionPlayers.csv file that has historical batting statistics. Only those statistics should be used to provide further context and influence the GenAI decision whether someone should join the Hall of Fame.  

In [5]:
// Use Baseball statistics to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 3000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Only use the historical baseball statistics to make your decision. 
Do NOT use web search results or machine learning models.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

To determine if Mike Trout should make the Hall of Fame based solely on historical baseball statistics, let’s review the traditional benchmarks and compare them to Trout’s career achievements:

### Hall of Fame Benchmarks for Position Players
- **Batting Average:** Historically, .300+ is elite, but many HOFers are below this.
- **Home Runs:** 500 is a famous “automatic” number, but many get in with fewer.
- **Hits:** 3,000 is a milestone, but not required.
- **RBI:** 1,500+ is typical, but context matters.
- **Awards:** MVPs, All-Star selections, and Gold Gloves add to a player’s case.
- **Longevity and Peak:** Consistency and dominance over several years are important.
- **Other Factors:** OBP, SLG, WAR (Wins Above Replacement) are modern evaluative stats.

### Mike Trout’s Career Stats (as of end of 2023)
- **Batting Average:** .301
- **Home Runs:** 368
- **Hits:** 1,624
- **RBI:** 940
- **Stolen Bases:** 206
- **All-Star Appearances:** 11
- **MVP Awards:** 3 (plus several top-2 fini

### Step 6 - Hall of Fame Decision - Only using a Machine Learning Model

> "The goal of probability is not to make precise predictions, but to make more accurate predictions than you would by chance."
>
> -- <cite>John Allen Paulos (famous mathmatecian and author)</cite> 

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. 

**Notice the instruction in the prompt is to only use the Machine Learning probability model.** Executing the code below will use a single probability number from a Machine Learning model to provide the intelligence for the GenAI to make the decision.  

In [6]:
// Use Machine Learning Prediction to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 3000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the Hall of Fame? 
Only use the machine learning probability for Hall Of Fame Induction to make your decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the machine learning probability, Mike Trout has a 94.5% chance of being inducted into the Hall of Fame. I recommend that Mike Trout should be inducted into the Hall of Fame.

### Step 7 - Hall of Fame Decision - Using all of the Available Tools (functions)

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall Of Fame. 

**Notice the instruction in the prompt is to use all of the tools available.** The example below will gather intelligence from: The Internet (Bing Service), Historical Statistics and Machine Learning model. 

In [7]:
// Use all of the available tools (functions) to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 3000, 
    Temperature = 0.4, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
You must cite all the sources using [number] notation of each URL after the reference in order.
The sources should appear in order on the bottom of the decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on a comprehensive review of Mike Trout’s career statistics, machine learning Hall of Fame probability, and current expert analysis, I strongly recommend that Mike Trout should be inducted into the Baseball Hall of Fame.

**Key Points Supporting Induction:**

1. **Statistical Excellence:**  
   - 13 years played, .301 career batting average, .582 slugging percentage, 368 home runs, 940 RBI, 206 stolen bases, and 1,624 hits as of 2023.
   - 11 All-Star appearances and 24 major player awards.
   - Trout’s advanced metrics and all-around game are among the most elite of his era, with his WAR (Wins Above Replacement) consistently ranking at the top of MLB[6].

2. **Probability of Induction:**  
   - According to machine learning models based on Hall of Fame voting trends and player statistics, Trout has a 94.5% probability of being inducted.

3. **Expert and Media Consensus:**  
   - Most baseball analysts and major publications consider Trout a “lock” for the Hall of Fame, citing hi

### Step 8 - Hall of Fame Decision - Adding Chain Of Thought Advanced Reasoning

Execute the cell below to have GenAI perform a decision recommendation on whether Mike Trout should make the baseball Hall of Fame.

The Prompt Instructions below include a deep reasoning framework on howto approach this problem, self-reflect and try different approaches. As was shown above the GenAI model has access to all of the tools used above (internet, historical data, historical baseball statistics).  

Contrast the execution results below with the previous results in this module. The GenAI model just doesn't gather intelligence and cite it when making a decision; the GenAI model now includes a deep reasoning process on how it has constructed the decision framework to arrive at the final decision.  Notice the outlined steps, the functions the GenAI model intends to call, scores itself etc. as part of the decision framework.  

In [8]:
// Use all of the available tools (functions) to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 3000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
You will be provided a question below.  
Begin by enclosing all thoughts within <thinking> tags, exploring multiple angles and approaches.
Break down the solution into clear steps within <step> tags. Start with a 20-step budget, requesting more for complex problems if needed.
Use <count> tags after each step to show the remaining budget. Stop when reaching 0.
Continuously adjust your reasoning based on intermediate results and reflections, adapting your strategy as you progress.
Regularly evaluate progress using <reflection> tags. Be critical and honest about your reasoning process.
Assign a quality score between 0.0 and 1.0 using <reward> tags after each reflection. Use this to guide your approach:

0.8+: Continue current approach
0.5-0.7: Consider minor adjustments
Below 0.5: Seriously consider backtracking and trying a different approach

If unsure or if reward score is low, backtrack and try a different approach, explaining your decision within <thinking> tags.
For mathematical problems, show all work explicitly using LaTeX for formal notation and provide detailed proofs.
Explore multiple solutions individually if possible, comparing approaches in reflections. Use thoughts as a scratchpad, writing out all calculations and reasoning explicitly.
Synthesize the final answer within <answer> tags, providing a clear, concise summary.
Conclude with a final reflection on the overall solution, discussing effectiveness, challenges, and solutions. Assign a final reward score.

Question:
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

<thinking>
To determine if Mike Trout should make the Hall of Fame, I need to comprehensively evaluate his career achievements, statistics, awards, and historical context. I will:
- Retrieve his up-to-date baseball statistics and accolades.
- Assess how these compare to typical Hall of Fame inductees.
- Use available predictive tools to estimate his induction probability.
- Weigh qualitative factors (e.g., impact, legacy).
- Synthesize all information for a recommendation.

I will use the BaseballHallOfFamePlugin to gather his stats and get a machine learning-based probability of induction. Throughout, I’ll critically reflect, adjust if needed, and ensure transparency in reasoning.
</thinking>

<step>
Step 1: Retrieve Mike Trout's comprehensive career statistics and accolades using the plugin.
</step>
<count>19</count>

<step>
Step 2: Obtain a machine learning-based probability of Mike Trout being inducted based on his statistics.
</step>
<count>18</count>

<step>
Step 3: Search for we

### Step 9 - Understanding Auto Function Calling Behavior  

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [9]:
// Method for printing the functions metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: BaseballHallOfFamePlugin
  Function Name: GetPlayerProbabilityOfHallOfFame
  Function Description: Retrieves a Machine Learning probability of a player being Inducted to Hall Of Fame using a the player's baseball statistics.
    Function Parameters:
      - baseballStatistics: The statistics of the baseball player.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetBaseballPlayerStats
  Function Description: Retrieves baseball statistics for a given baseball player from historical data.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetWebSearchResults
  Function Description: Retrieves the web search results with identified URL sources for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

